In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from python_speech_features import mfcc
import scipy.io.wavfile as wav
from tempfile import TemporaryFile
import os
import pickle
import random 
import operator
import math

In [2]:
audio_features=[]
labels=[]

In [3]:
def extract_features(file_path):
    (rate, sig)=wav.read(file_path)
    mfcc_feat=mfcc(sig, rate, winlen=0.020, appendEnergy = False)
    return  mfcc_feat

In [4]:
print(extract_features("data/blues/blues.00001.wav"))

[[ 52.32772636  12.27974663 -14.05910548 ...  -0.2150729   10.56329052
   -7.87888343]
 [ 54.98108456  16.45550208 -19.12696735 ... -11.48633547   3.45238194
   -7.25151242]
 [ 62.43432252   2.47227707  -2.81401672 ... -10.43204984  -2.66200942
    2.29782969]
 ...
 [ 77.41351398  -3.69330514   2.91007987 ... -10.29548132   3.28747789
    8.65780958]
 [ 77.4757659   -5.02653026   5.39881623 ... -10.59178905   0.9218981
    5.95638195]
 [ 76.60870935  -3.13448805   5.46082437 ... -13.14580229  -0.44330338
    8.73963484]]


In [5]:
folder_path="data/"
genres=os.listdir(folder_path)
d={}
count=0
for folder in genres:
    subfolder_path=os.path.join(folder_path, folder)
    lists=os.listdir(subfolder_path)
    for j in lists:
        song_path=os.path.join(subfolder_path, j)
        audio_features.append(extract_features(song_path))
        if (folder not in d):
            d[folder]=count
            count+=1
         
        labels.append(d[folder])


In [6]:
print(labels)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]


In [7]:
dict_len=len(d)
dict_len

5

In [8]:
print(audio_features[4].shape)

(2994, 13)


In [9]:
audio_features=np.array(audio_features)
labels=np.array(labels)

C:\Users\DELL\AppData\Local\Temp\ipykernel_24736\3561159992.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  audio_features=np.array(audio_features)


In [10]:
new_shape=(2990,13)

new_audio_features=[]
for i in audio_features:
    new_audio_features.append(i[:new_shape[0], :new_shape[1]].reshape(new_shape))

In [11]:
new_audio_features=np.array(new_audio_features)

In [12]:
new_audio_features.shape

(51, 2990, 13)

In [13]:
labels[0].shape

()

In [14]:
from keras.utils.np_utils import to_categorical

In [15]:
labels=to_categorical(labels)

In [16]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test=train_test_split(new_audio_features,labels, test_size=0.2, random_state=0)

In [17]:
x_train.shape

(40, 2990, 13)

In [18]:
x_test.shape

(11, 2990, 13)

In [19]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

In [20]:
model=models.Sequential([
    layers.Conv2D(64, kernel_size=(3,3), activation="relu", input_shape=(new_audio_features.shape[1], new_audio_features.shape[2],1)),
    layers.MaxPool2D(pool_size=2,strides=2 ),
    layers.Conv2D(32, kernel_size=(3,3), activation="relu",),
    layers.MaxPool2D(pool_size=2,strides=2 ),
    layers.Flatten(),
    layers.Dense(units=128, activation="relu"),
    layers.Dense(units=64, activation="relu"),
    layers.Dense(units=dict_len, activation="softmax")
])

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 2988, 11, 64)      640       
                                                                 
 max_pooling2d (MaxPooling2  (None, 1494, 5, 64)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 1492, 3, 32)       18464     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 746, 1, 32)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 23872)             0         
                                                                 
 dense (Dense)               (None, 128)               3

In [26]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [27]:
model.fit(x_train, y_train , epochs=10)

Epoch 1/10
2/2 [==============================] - 3s 280ms/step - loss: 0.9033 - accuracy: 0.8250
Epoch 2/10
2/2 [==============================] - 2s 347ms/step - loss: 0.1641 - accuracy: 0.9500
Epoch 3/10
2/2 [==============================] - 1s 281ms/step - loss: 0.6237 - accuracy: 0.7500
Epoch 4/10
2/2 [==============================] - 1s 276ms/step - loss: 0.4311 - accuracy: 0.8000
Epoch 5/10
2/2 [==============================] - 1s 271ms/step - loss: 0.3530 - accuracy: 0.8750
Epoch 6/10
2/2 [==============================] - 1s 287ms/step - loss: 0.2387 - accuracy: 0.9750
Epoch 7/10
2/2 [==============================] - 1s 252ms/step - loss: 0.1869 - accuracy: 0.9750
Epoch 8/10
2/2 [==============================] - 1s 239ms/step - loss: 0.1128 - accuracy: 1.0000
Epoch 9/10
2/2 [==============================] - 1s 228ms/step - loss: 0.0684 - accuracy: 1.0000
Epoch 10/10
2/2 [==============================] - 1s 231ms/step - loss: 0.0334 - accuracy: 1.0000


In [34]:
test_data=extract_features("data/test_set/blues.00016.wav")

In [35]:
test_data=test_data[:new_shape[0], :new_shape[1]].reshape(new_shape)

In [36]:
test_data.shape

(2990, 13)

In [38]:
test_set=[]
test_set.append(test_data)

In [39]:
test_set=np.array(test_set)

In [40]:
test_set.shape

(1, 2990, 13)

In [42]:
y_pred=model.predict(test_set)

1/1 [==============================] - 0s 49ms/step


In [43]:
np.argmax(y_pred)

3

In [49]:
np.argmax(labels[1])

0

In [50]:
y_pred

array([[0.48287728, 0.10909961, 0.05564116, 0.8173537 , 0.07006252]],
      dtype=float32)